In [25]:
import pandas as pd
import glob
import numpy as np

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
columns_to_load = [
     'location_key'
    , 'date'
    , 'place_id'
    , 'wikidata_id'
    , 'datacommons_id'
    , 'country_code'
    , 'country_name'
    , 'subregion1_code'
    , 'subregion1_name'
    , 'subregion2_code'
    , 'subregion2_name'
    , 'new_confirmed'
    , 'new_deceased'
    , 'cumulative_confirmed'
    , 'cumulative_deceased'
    , 'new_persons_fully_vaccinated'
    , 'cumulative_persons_fully_vaccinated'
    , 'population'
    , 'population_male'
    , 'population_female'
    , 'population_age_00_09'
    , 'population_age_10_19'
    , 'population_age_20_29'
    , 'population_age_30_39'
    , 'population_age_40_49'
    , 'population_age_50_59'
    , 'population_age_60_69'
    , 'population_age_70_79'
    , 'population_age_80_and_older'
    , 'openstreetmap_id'
    , 'latitude'
    , 'longitude'
    , 'area_sq_km'
    , 'life_expectancy'
    , 'mobility_retail_and_recreation'
    , 'mobility_grocery_and_pharmacy'
    , 'mobility_parks'
    , 'mobility_transit_stations'
    , 'mobility_workplaces'
    , 'mobility_residential'
    , 'average_temperature_celsius'
    , 'minimum_temperature_celsius'
    , 'maximum_temperature_celsius'
    , 'rainfall_mm'
    , 'dew_point'
    , 'relative_humidity'
]

In [3]:
# Directory path to your CSV files
directory_path = "../Test CSVs/"  # Example directory path

# Initialize an empty DataFrame to store data from all CSV files
full_df = pd.DataFrame(columns=columns_to_load)

# Loop over the list of CSV files
for file in glob.glob(directory_path + '*.csv'):
    # Read the CSV file without specifying columns to ensure we don't miss any data
    df = pd.read_csv(file)
    
    # Ensure all desired columns are present, add them with NaN values if they are missing
    for column in columns_to_load:
        if column not in df.columns:
            df[column] = np.nan

    # Reorder and select only the desired columns to maintain consistency
    df = df[columns_to_load]
    
    # Append the contents of the file to the full DataFrame
    full_df = pd.concat([full_df, df], ignore_index=True)

# Optionally, reset the index of the final DataFrame
full_df.reset_index(drop=True, inplace=True)



/tmp/ipykernel_1903/2111723118.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, df], ignore_index=True)
/tmp/ipykernel_1903/2111723118.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, df], ignore_index=True)


In [42]:
# Data Prep
freq_df = full_df.copy()
freq_df['date'] = pd.to_datetime(freq_df['date'])
freq_df['quarter'] = freq_df['date'].dt.to_period('Q').dt.start_time
freq_df['month'] = freq_df['date'].dt.to_period('M').dt.to_timestamp()
freq_df['week'] = freq_df['date'] - pd.to_timedelta(freq_df['date'].dt.weekday, unit='d')
freq_df['county_name'] = freq_df['subregion2_name'] + ', ' + freq_df['subregion1_name']

columns_to_keep = [
    'county_name', 'location_key', 'quarter', 'month', 'week', 'date',
    'new_confirmed', 'new_deceased', 'cumulative_confirmed', 'cumulative_deceased',
    'new_persons_fully_vaccinated', 'cumulative_persons_fully_vaccinated',
    'population', 'population_male', 'population_female',
    'population_age_00_09', 'population_age_10_19', 'population_age_20_29',
    'population_age_30_39', 'population_age_40_49', 'population_age_50_59',
    'population_age_60_69', 'population_age_70_79', 'population_age_80_and_older',
    'area_sq_km',
    'life_expectancy',
    'average_temperature_celsius', 'minimum_temperature_celsius',
    'maximum_temperature_celsius', 'rainfall_mm',
    'relative_humidity'
]

freq_df = freq_df[columns_to_keep]

# Convert object types to float64 where applicable
for column in freq_df.select_dtypes(include=['object']).columns:
    try:
        freq_df[column] = freq_df[column].astype(float)
    except ValueError:
        print(f"Conversion failed for column: {column}")
    

aggregations = {
    'new_confirmed': 'sum',
    'new_deceased': 'sum',
    'cumulative_confirmed': 'max',
    'cumulative_deceased': 'max',
    'new_persons_fully_vaccinated': 'sum',
    'cumulative_persons_fully_vaccinated': 'max',
    'population': 'max',
    'population_male': 'max',
    'population_female': 'max',
    'population_age_00_09': 'max',
    'population_age_10_19': 'max',
    'population_age_20_29': 'max',
    'population_age_30_39': 'max',
    'population_age_40_49': 'max',
    'population_age_50_59': 'max',
    'population_age_60_69': 'max',
    'population_age_70_79': 'max',
    'population_age_80_and_older': 'max',
    'area_sq_km': 'max',
    'life_expectancy': 'max',
    'average_temperature_celsius': 'mean',
    'minimum_temperature_celsius': 'mean',
    'maximum_temperature_celsius': 'mean',
    'rainfall_mm': 'sum',
    'relative_humidity': 'mean'
}

change_columns = [
    'new_confirmed',
    'new_deceased',
    'cumulative_confirmed',
    'cumulative_deceased',
    'new_persons_fully_vaccinated',
    'cumulative_persons_fully_vaccinated',
    'average_temperature_celsius',
    'minimum_temperature_celsius',
    'maximum_temperature_celsius',
    'rainfall_mm',
    'relative_humidity'
]

# Group and aggregate for weekly, monthly, quarterly
weekly_df = freq_df.groupby(['county_name', 'week']).agg(aggregations).reset_index()
monthly_df = freq_df.groupby(['county_name', 'month']).agg(aggregations).reset_index()
quarterly_df = freq_df.groupby(['county_name', 'quarter']).agg(aggregations).reset_index()

# Format descriptive columns
weekly_df['description'] = weekly_df['county_name'] + " - Week of " + weekly_df['week'].dt.strftime('%Y-%m-%d')
monthly_df['description'] = monthly_df['county_name'] + " - Month of " + monthly_df['month'].dt.strftime('%Y-%m')
quarterly_df['description'] = quarterly_df['county_name'] + " - Quarter of " + quarterly_df['quarter'].dt.strftime('%Y-Q%q')

# Calculate the percentage change within each group
for df in [weekly_df, monthly_df, quarterly_df]:
    # Ensure the DataFrame is sorted by date to correctly apply pct_change
    df.sort_values(by=['county_name', 'week' if 'week' in df.columns else 'month' if 'month' in df.columns else 'quarter'], inplace=True)
    df.set_index(['county_name', df.columns[1]], inplace=True)  # Set index as county_name and the time period
    for col in change_columns:
        df[f'{col}_pct_change'] = df.groupby('county_name')[col].pct_change(fill_method=None).multiply(100)

    # Reset index after calculations
    df.reset_index(inplace=True)


Conversion failed for column: county_name
Conversion failed for column: location_key


In [ ]:
# Function to apply binning
def apply_binning(df):
    # Define bins and labels for new_confirmed as an example
    bins = [0, 10, 100, 1000, 10000, float('inf')]
    labels = ['1-10', '11-100', '101-1000', '1001-10000', '>10000']
    df['new_confirmed_bins'] = pd.cut(df['new_confirmed'], bins=bins, labels=labels, right=False)

    # Apply similar binning to other columns as needed, adjust bins and labels accordingly
    # Example for average_temperature_celsius
    temp_bins = [df['average_temperature_celsius'].min(), 0, 15, 25, 35, df['average_temperature_celsius'].max()]
    temp_labels = ['<0°C', '0-15°C', '15-25°C', '25-35°C', '>35°C']
    df['average_temperature_celsius_bins'] = pd.cut(df['average_temperature_celsius'], bins=temp_bins, labels=temp_labels, right=False)

    # Apply this to other relevant columns
    # Continue for other metrics like rainfall_mm, relative_humidity, etc., customizing bins as needed

# Apply binning to each DataFrame
for dataframe in [weekly_df, monthly_df, quarterly_df]:
    apply_binning(dataframe)

In [43]:
sample_df = weekly_df.sample(n=10)
sample_df

,county_name,week,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,population,population_male,population_female,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,area_sq_km,life_expectancy,average_temperature_celsius,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,relative_humidity,description,new_confirmed_pct_change,new_deceased_pct_change,cumulative_confirmed_pct_change,cumulative_deceased_pct_change,new_persons_fully_vaccinated_pct_change,cumulative_persons_fully_vaccinated_pct_change,average_temperature_celsius_pct_change,minimum_temperature_celsius_pct_change,maximum_temperature_celsius_pct_change,rainfall_mm_pct_change,relative_humidity_pct_change
43355,"Rock County, Wisconsin",2020-11-09,1111.0,6.0,7995.0,62.0,0.0,NaN,162205.0,79882.0,82323.0,20120.0,21936.0,19637.0,20345.0,20030.0,23256.0,19315.0,11024.0,6542.0,1881.0,NaN,7.437302,2.680159,13.956349,16.482483,76.271337,"Rock County, Wisconsin - Week of 2020-11-09",19.848975,-25.000000,16.138873,10.714286,NaN,NaN,-33.895317,-36.866709,-30.808577,inf,24.112470
2202,"Ashe County, North Carolina",2021-05-17,16.0,0.0,2143.0,41.0,268.0,8991.0,26787.0,13175.0,13612.0,2502.0,2770.0,2491.0,2756.0,3258.0,4054.0,4438.0,3008.0,1510.0,1105.0,77.483333,18.185450,11.179189,25.315520,0.878417,65.502855,"Ashe County, North Carolina - Week of 2021-05-17",-36.000000,NaN,0.752233,0.000000,72.903226,3.072337,50.371876,102.673062,32.398953,-90.844912,2.134732
645,"Alameda County, California",2021-06-21,511.0,4.0,90124.0,1287.0,24861.0,1023762.0,1660196.0,816390.0,843806.0,194444.0,189121.0,239581.0,270207.0,231707.0,216422.0,171834.0,91548.0,55332.0,2127.0,80.845858,18.369048,14.954762,23.292857,0.000000,66.827382,"Alameda County, California - Week of 2021-06-21",149.268293,-166.666667,0.570230,0.941176,-24.993212,2.488835,-13.295121,-0.758411,-19.366450,NaN,22.641500
15566,"Fond du Lac County, Wisconsin",2021-09-06,245.0,1.0,15193.0,151.0,664.0,48183.0,102417.0,50433.0,51984.0,11729.0,12923.0,12039.0,12600.0,12228.0,15267.0,13241.0,7411.0,4979.0,1983.0,NaN,18.374603,12.793651,24.781746,3.690257,72.461336,"Fond du Lac County, Wisconsin - Week of 2021-0...",-21.221865,-66.666667,1.639015,0.666667,19.210054,1.397336,-4.227683,-12.195653,4.403504,46.354326,-3.163989
31281,"Mecklenburg County, North Carolina",2020-10-12,1322.0,11.0,31688.0,377.0,0.0,NaN,1078128.0,518115.0,560013.0,144047.0,140261.0,165091.0,171435.0,152667.0,134159.0,95822.0,48537.0,26109.0,1415.0,78.516327,16.950794,11.238889,23.707143,32.646257,75.886097,"Mecklenburg County, North Carolina - Week of 2...",44.956140,266.666667,4.353553,3.005464,NaN,NaN,-7.593130,-10.413109,-6.295878,-13.912545,-4.839669
42751,"Rio Blanco County, Colorado",2020-03-02,0.0,0.0,NaN,NaN,0.0,NaN,6352.0,3257.0,3095.0,838.0,903.0,762.0,859.0,694.0,890.0,769.0,411.0,226.0,8347.0,78.950000,0.926191,-6.313492,9.886508,0.444500,59.825364,"Rio Blanco County, Colorado - Week of 2020-03-02",NaN,NaN,NaN,NaN,NaN,NaN,-121.944343,-44.653167,147.752586,-76.091083,1.602718
17947,"Graham County, North Carolina",2021-01-18,16.0,0.0,511.0,12.0,0.0,NaN,8524.0,4240.0,4284.0,879.0,1042.0,914.0,907.0,967.0,1178.0,1229.0,895.0,513.0,781.0,75.233333,4.299559,-0.963845,10.845767,4.171244,62.793586,"Graham County, North Carolina - Week of 2021-0...",-27.272727,-100.000000,3.232323,0.000000,NaN,NaN,76.809566,-49.878480,26.980732,109.052323,-17.073630
40217,"Pitt County, North Carolina",2020-08-03,254.0,0.0,2020.0,11.0,0.0,NaN,178607.0,84041.0,94566.0,21052.0,25856.0,37006.0,21677.0,20273.0,20048.0,18004.0,9429.0,5262.0,1696.0,77.713793,25.609524,21.831129,31.498060,87.215739,85.500872,"Pitt County, North Carolina - Week of 2020-08-03",-13.605442,-100.000000,14.382786,0.000000,NaN,NaN,-7.163818,-5.314027,-7.615270,178.067362,8.252

In [44]:
# Print each column name
for column in weekly_df.columns:
    print(column)

county_name
week
new_confirmed
new_deceased
cumulative_confirmed
cumulative_deceased
new_persons_fully_vaccinated
cumulative_persons_fully_vaccinated
population
population_male
population_female
population_age_00_09
population_age_10_19
population_age_20_29
population_age_30_39
population_age_40_49
population_age_50_59
population_age_60_69
population_age_70_79
population_age_80_and_older
area_sq_km
life_expectancy
average_temperature_celsius
minimum_temperature_celsius
maximum_temperature_celsius
rainfall_mm
relative_humidity
description
new_confirmed_pct_change
new_deceased_pct_change
cumulative_confirmed_pct_change
cumulative_deceased_pct_change
new_persons_fully_vaccinated_pct_change
cumulative_persons_fully_vaccinated_pct_change
average_temperature_celsius_pct_change
minimum_temperature_celsius_pct_change
maximum_temperature_celsius_pct_change
rainfall_mm_pct_change
relative_humidity_pct_change


### Deciding on Bin sizes